In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Charger les données à partir d'un fichier CSV
df_train = pd.read_csv('train.csv')
df = df_train

# Extraire la marque de la colonne "Name"
df['Name_short'] = df['Name'].str.split(' ').str[0]

# Convertir les prix en unités de lakhs en roupies indiennes
conversion_rate = 87
lakh_conversion = 100000
df['Price_Rupie'] = df['Price'] * lakh_conversion

# Convertir les prix en euros
df['Price_EUR'] = df['Price_Rupie'] / conversion_rate
df['Price_EUR'] = df['Price_EUR'].round(2)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2('Max Price Change by Brand and Fuel Type'),
    html.Div([
        html.Label('Fuel Type:'),
        dcc.Dropdown(
            id='fuel-dropdown',
            options=[{'label': x, 'value': x} for x in df['Fuel_Type'].unique()],
            value='All'
        )
    ]),
    html.Div([
        html.Label('Start Year:'),
        dcc.Dropdown(
            id='start-year-dropdown',
            options=[{'label': x, 'value': x} for x in df['Year'].unique()],
            value=df['Year'].min()
        )
    ]),
    html.Div([
        html.Label('End Year:'),
        dcc.Dropdown(
            id='end-year-dropdown',
            options=[{'label': x, 'value': x} for x in df['Year'].unique()],
            value=df['Year'].max()
        )
    ]),
    html.Div([
        html.Label('Brand:'),
        dcc.Dropdown(
            id='brand-dropdown',
            options=[{'label': x, 'value': x} for x in df['Name_short'].unique()],
            value='All'
        )
    ]),
    dcc.Graph(id='price-change-chart')
])

@app.callback(
    Output('price-change-chart', 'figure'),
    Input('fuel-dropdown', 'value'),
    Input('start-year-dropdown', 'value'),
    Input('end-year-dropdown', 'value'),
    Input('brand-dropdown', 'value')
)
def update_chart(fuel_type, start_year, end_year, brand):
    mask = (df['Fuel_Type'] == fuel_type) if fuel_type != 'All' else df['Fuel_Type'].isin(df['Fuel_Type'].unique())
    mask &= (df['Year'] >= start_year) & (df['Year'] <= end_year)
    mask &= (df['Name_short'] == brand) if brand != 'All' else df['Name_short'].isin(df['Name_short'].unique())

    df_filtered = df[mask]

    # Calculer l'évolution du prix pour chaque année et chaque marque
    price_change = df_filtered.groupby(['Year', 'Name_short'])['Price_EUR'].apply(lambda x: x.pct_change()).reset_index()
    price_change = price_change.rename(columns={'Price_EUR': 'Price_Change'})
    price_change = price_change[~price_change['Price_Change'].isnull()]

    # Sélectionner la plus grande évolution de prix pour chaque marque
    max_price_change = price_change.loc[price_change.groupby('Name_short')['Price_Change'].idxmax()]

    # Tracer un graphique à barres montrant la plus grande évolution de prix pour chaque marque
    fig = px.bar(max_price_change, x='Name_short', y='Price_Change', title='Max Price Change by Brand and Fuel Type')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

